In [2]:
import numpy as np
import csv


communities = [25,25,25,25]
p_matrix = np.array([[0.75, 0.05, 0.05, 0.05],
                     [0.05, 0.6, 0.1, 0.1],
                     [0.05, 0.1, 0.6, 0.1],
                     [0.05, 0.1, 0.1, 0.7]])

def generate_poison_sbm(communities, p_matrix, k, output_csv, rate=1.0):
    """
    参数:
      communities: list，社区结构，每个元素代表该社区中的节点数，例如 [25, 25, 25, 25]
      p_matrix: numpy 数组，形状为 (n, n)，其中 n 为社区数。p_matrix[i,j] 表示社区 i 与社区 j 之间的链接概率
      k: int，要添加的边数
      rate: float，泊松过程的速率参数（默认为1.0），用于采样边的到达时间（指数分布的参数）
    
    返回:
      edges: list，每个元素为一个元组 (u, v, t)，其中 u, v 分别是边的两个节点的索引，t 是边产生的时间戳
    """
    total_nodes = sum(communities)
    
    # 为每个社区构造节点列表，节点编号从 0 到 total_nodes-1
    community_nodes = []
    start = 0
    for c in communities:
        community_nodes.append(np.arange(start, start + c))
        start += c

    n = len(communities)
    # 预先计算每对社区的权重：权重 = (社区 i 的比例) * (社区 j 的比例) * p_matrix[i,j]
    weights = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            weights[i, j] = (communities[i] / total_nodes) * (communities[j] / total_nodes) * p_matrix[i, j]
    # 将二维矩阵展平成一维，并归一化得到概率分布
    flat_weights = weights.flatten()
    flat_weights = flat_weights / flat_weights.sum()
    
    # 构造社区对的索引集合，对应 (i,j) 对应位置为 i*n+j
    community_pair_indices = np.arange(n * n)
    
    edges = []
    current_time = 0.0
    for _ in range(k):
        # 采样边的到达间隔（指数分布）
        inter_arrival = np.random.exponential(1.0 / rate)
        current_time += inter_arrival
        
        # 根据社区对分布采样一个社区对
        chosen_index = np.random.choice(community_pair_indices, p=flat_weights)
        i = chosen_index // n  # 社区 i
        j = chosen_index % n   # 社区 j
        
        # 分别从社区 i 和 j 中随机选取一个节点
        u = np.random.choice(community_nodes[i])
        v = np.random.choice(community_nodes[j])
        
        # 将边 (u, v) 及其时间戳加入结果中
        edges.append((u, v, current_time))
    
    # 如果提供了 output_csv 参数，则将边保存为 CSV 文件
    if output_csv is not None:
        with open(output_csv, mode='w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['u', 'i', 'ts'])
            writer.writerows(edges)
    return edges

# 示例使用
if __name__ == "__main__":
    communities = [25,25,25,25]
    p_matrix = np.array([[0.75, 0.05, 0.05, 0.05],
                         [0.05, 0.6,  0.1,  0.1],
                         [0.05, 0.1,  0.6,  0.1],
                         [0.05, 0.1,  0.1,  0.7]])
    k = 1000
    for i in range(1):
        edges = generate_poison_sbm(communities, p_matrix, k, output_csv=f"SBM-r1-{i}.csv", rate=1.0)

In [ ]:
import pandas as pd
import argparse

def reindex_nodes(input_csv, output_csv):
    # 读取原始文件
    df = pd.read_csv(input_csv)

    # 提取所有唯一的节点
    nodes = pd.unique(df[['u', 'v']].values.ravel())
    
    # 建立节点映射表：原节点ID -> 新ID
    node_mapping = {node: idx for idx, node in enumerate(nodes)}

    # 替换 u 和 v 为新的编号
    df['u'] = df['u'].map(node_mapping)
    df['v'] = df['v'].map(node_mapping)

    # 添加 idx 列（从0开始）
    df.insert(0, 'idx', range(len(df)))

    # 保存新的 CSV 文件
    df.to_csv(output_csv, index=False)
    print(f"转换完成，保存到：{output_csv}")

    # 如果你也想保存映射表，可以取消注释以下两行
    # pd.Series(node_mapping).to_csv('node_mapping.csv', header=False)
    # print("节点映射表已保存为 node_mapping.csv")


reindex_nodes("data/SBM-r1-0.csv", "data/SBM-r1-0-reindexed.csv")

In [6]:
from CTDNE.CTDNE.ctdne import CTDNE
import networkx as nx 
import numpy as np
import csv
def csv_to_graph(file_name):
    G = nx.MultiGraph()
    # 打开并读取 CSV 文件
    with open(f'data/{file_name}.csv', "r", newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            node1 = row["u"]
            node2 = row["v"]
            timestamp = float(row["timestamp"])
            G.add_edge(node1, node2, time=timestamp)
    return G

name = "SBM-r1-0-reindexed"
graph = csv_to_graph(name)
CTDNE_model = CTDNE(graph, dimensions=64, walk_length=30, num_walks=200, workers=4)
# Embed nodes
model = CTDNE_model.fit(window=10, min_count=1, batch_words=4)

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 58.51it/s]


In [3]:
import numpy as np

# 假设模型中有 num_nodes 个节点，可以通过 len(model.wv) 获得
num_nodes = len(model.wv)

# 将所有节点的嵌入向量整合成一个矩阵，每一行代表一个节点的嵌入
embeddings = np.array([model.wv[i] for i in range(num_nodes)])

# 保存为 npy 文件
np.save('node_embeddings.npy', embeddings)